In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network

In [2]:
N_NODES = 20
N_HUBS = 3
MIN_CONNECTS = 2
COLOR_1 = "#DE0100"
COLOR_2 = "#1405BD"
P_COLOR_1 = 0.3
P_FRIENDS_ACROSS_COLORS = 0.5
STARTING_EDGE_WEIGHT_INDIV = 0.5
STARTING_EDGE_WEIGHT_HUB = 1
STARTING_FRIENDS = 2
STARTING_HUBS = 2

In [3]:
G = nx.barabasi_albert_graph(N_NODES, MIN_CONNECTS)

# get id's of hub nodes
edge_counts = []
for i in range(N_NODES):
    num_edges = G.degree(i)
    edge_counts.append(num_edges)
    
edge_counts = sorted(edge_counts)
hub_edge_counts = edge_counts[-N_HUBS:]

num_hubs = 0
hub_color_count = {COLOR_1 : 0, COLOR_2 : 0}
for i in range(N_NODES):
    current_node = G.nodes[i]

    # create hubs of each color in equal number
    if G.degree(i) in hub_edge_counts and num_hubs < N_HUBS:
        num_hubs += 1
        current_node["size"] = 20
        current_node["is_hub"] = True
        current_node["hover_data"] = "Media Hub"
        if hub_color_count[COLOR_1] < hub_color_count[COLOR_2]:
            color = COLOR_1
            hub_color_count[COLOR_1] += 1
        else:
            color = COLOR_2
            hub_color_count[COLOR_2] += 1
            
    # create individuals of a color from chosen proportion
    else:
        color = np.random.choice([COLOR_1, COLOR_2], p = [P_COLOR_1, 1 - P_COLOR_1])
    current_node["color"] = color

#net = Network(notebook = True)
#net.from_nx(G)
#net.show("g.html")

## Generate graph manually

### Generate nodes

In [4]:
G = nx.Graph()

current_n_hubs = 0
hub_edges = dict()
nodes = []

prev_hub_color = COLOR_1 if P_COLOR_1 < 0.5 else COLOR_2
color1_indivs = []
color2_indivs = []
color1_hubs = []
color2_hubs = []

for i in range(N_NODES):
    new_node = (i, dict())
    new_node_atts = new_node[1]

    # if new node is a hub (first N nodes are hubs)
    if i < N_HUBS:
        new_node_atts["size"] = 20
        new_node_atts["color"] = COLOR_1 if prev_hub_color == COLOR_2 else COLOR_2
        prev_hub_color = COLOR_2 if prev_hub_color == COLOR_1 else COLOR_1
        if new_node_atts["color"] == COLOR_1:
            color1_hubs.append(i)
        else: 
            color2_hubs.append(i)
        current_n_hubs += 1

    # if not a hub, pick color according to given proportion
    else:
        color = np.random.choice([COLOR_1, COLOR_2], p = [P_COLOR_1, 1 - P_COLOR_1])
        new_node_atts["color"] = color
        if color == COLOR_1:
            color1_indivs.append(i)
        else:
            color2_indivs.append(i)
    nodes.append(new_node)     
    
# instantiate nodes in graph
G.add_nodes_from(nodes)

In [5]:
# create edges between nodes
for i in range(N_NODES):
    current_node = G.nodes()[i]
    current_node_color = current_node["color"]
    
    # connect to STARTING_FRIENDS individuals and STARTING_HUBS hubs
    for j in range(STARTING_FRIENDS + STARTING_HUBS):
        
        # skip hubs
        if i < N_HUBS:
            continue

        # connect to hubs before individuals
        # individuals start by consuming equal amounts of each color media
        # odd number starting hubs will have an extra of the node's color
        if j < STARTING_HUBS:
            if j % 2 == 0:
                color_hub = color1_hubs if current_node_color == COLOR_1 else color2_hubs
            else:
                color_hub = color2_hubs if current_node_color == COLOR_2 else color1_hubs

            # randomly select hub of desired color
            hub_id = np.random.choice(color_hub, 
                                      p = [1 / len(color_hub)] * len(color_hub))
            G.add_edge(i, hub_id, weight = STARTING_EDGE_WEIGHT_HUB)
            continue
                
    
        # make an edge between nodes
        connect_to = -1
        while connect_to != i:
            connect_to = np.random.randint(N_HUBS - 1, N_NODES - 1)
            if G.nodes()[connect_to]["color"] == current_node["color"]:
                
                # allow cross-color friends with probability P_FRIENDS_ACROSS_COLORS
                p_friends_across = np.random.uniform()
                if p_friends_across <= P_FRIENDS_ACROSS_COLORS:
                    G.add_edge(i, connect_to, weight = STARTING_EDGE_WEIGHT_INDIV)
                    # change connect_to to break out of while loop
                    connect_to = i + 1
                    continue
                    
                connect_to = -1
                continue
                
            else:
                # add age with STARTING_EDGE_WEIGHT_INDIV
                G.add_edge(i, connect_to, weight = STARTING_EDGE_WEIGHT_INDIV)

KeyboardInterrupt: 

### Display graph network

In [6]:
# display network graph
net2 = Network(notebook = True)
net2.from_nx(G)
net2.show("g2.html")

g2.html
